# FEATURE ENGINEERING

# INFO

El objetivo de este _notebook_ es realizar el proceso de _feature engineering_ de las variables del conjunto de datos.

In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../functions')
import warnings

In [2]:
def one_hot_encoder(df, nan_as_category = True):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns

En este notebook se expone la ingeniería de variables realizada sobre el conjunto de bases de datos proporcionados por *Home Credit*. 

La información proporcionada por cada una de las bases de datos es la siguiente:

- __application_{train|test}.csv__: 
    - Tabla principal donde se encuentran los datos de solicitud de crédito de los clientes.
    - Cada fila representa un préstamo.
    
    
- __bureau.csv__: 
    - Información de créditos anteriores de los clientes proporcionados por instituciones financieras.
    - Por cada crédito de la tabla principal existen tantas filas como créditos pasados reportados por la oficina de crédito (bureau) antes de la solicitud del crédito.
    
    
- __bureau_balance.csv__:
    - Balances mensuales de los créditos pasados en la oficina de crédito.
    - Cada fila corresponde con una mensualidad de los créditos pasados.
    
    
- __POS_CASH_balance.csv__: 
    - Instantaneas de los balances mensuales anteriores al momento del concesión del crédito e información histórica de créditos que los clientes adquirieron con *Home Credit*.
    
    
- __credit_card_balance.csv__: 
    - Instantaneas de los balances mensuales de las tarjetas de crédito que los clientes tienen con *Home Credit*.
    
    
-  __previous_application.csv__: 
    - Aplicaciones previas para créditos con *Home Credit* de clientes que tienen un préstamo en la tabla principal..
    
    
- __installments_payments.csv__: 
    - Historial de pagos de créditos anteriores con *Home Credit*.
    - Cada fila corresponde con cada pago efectuado o con cada pago no realizado en su fecha correspondiente.

Utilizando la información de todas las tables se agrega la información en una única base de datos mediante tranformaciones de las variables ya existentes.
Este proceso de ingenieria se realiza para la utilización de un modelo LightGBM, algoritmo basado en arboles de decisión.

### Transformaciones `application_train.csv`

In [3]:
data_path = "../data/application_train.csv"

In [4]:
datos = pd.read_csv(data_path)

Eliminamos la variable de género ya que no puede usarse como información a la hora de establecer el riesgo de impago de un cliente.

In [5]:
datos.drop('CODE_GENDER', axis=1, inplace=True)

In [6]:
# Variables categóricas binarias
for bin_feature in ['FLAG_OWN_CAR', 'FLAG_OWN_REALTY']:
        datos[bin_feature], uniques = pd.factorize(datos[bin_feature])

In [7]:
datos, cat_cols = one_hot_encoder(datos)

En la variable `DAYS_EMPLOYED` el valor 365.243 parece estár relacionado con falta de información (NA).

In [8]:
datos['DAYS_EMPLOYED'].replace(365243, np.nan, inplace= True)

In [9]:
# Nuevas variables (porcentajes)
datos['DAYS_EMPLOYED_PERC'] = datos['DAYS_EMPLOYED'] / datos['DAYS_BIRTH'] 
datos['INCOME_CREDIT_PERC'] = datos['AMT_INCOME_TOTAL'] / datos['AMT_CREDIT'] 
datos['INCOME_PER_PERSON'] = datos['AMT_INCOME_TOTAL'] / datos['CNT_FAM_MEMBERS'] 
datos['ANNUITY_INCOME_PERC'] = datos['AMT_ANNUITY'] / datos['AMT_INCOME_TOTAL'] 
datos['PAYMENT_RATE'] = datos['AMT_ANNUITY'] / datos['AMT_CREDIT']

### Transformaciones de `bureau.csv` y `bureau_balance.csv``

In [10]:
bureau = pd.read_csv('../data/bureau.csv')
bb = pd.read_csv('../data/bureau_balance.csv')

In [11]:
bb, bb_cat = one_hot_encoder(bb, nan_as_category=True)
bureau, bureau_cat = one_hot_encoder(bureau, nan_as_category=True)

In [12]:
# Agregaciones bureau_balance
bb_aggregations = {'MONTHS_BALANCE': ['min', 'max', 'size']}
for col in bb_cat:
    bb_aggregations[col] = ['mean']
bb_agg = bb.groupby('SK_ID_BUREAU').agg(bb_aggregations)
bb_agg.columns = pd.Index([e[0] + "_" + e[1].upper() for e in bb_agg.columns.tolist()])

# Junto bureau y bureau_balance
bureau = bureau.join(bb_agg, how='left', on='SK_ID_BUREAU')
bureau.drop(['SK_ID_BUREAU'], axis=1, inplace= True)
del bb, bb_agg

In [13]:
num_aggregations = {
    'DAYS_CREDIT': ['min', 'max', 'mean', 'var'],
    'DAYS_CREDIT_ENDDATE': ['min', 'max', 'mean'],
    'DAYS_CREDIT_UPDATE': ['mean'],
    'CREDIT_DAY_OVERDUE': ['max', 'mean'],
    'AMT_CREDIT_MAX_OVERDUE': ['mean'],
    'AMT_CREDIT_SUM': ['max', 'mean', 'sum'],
    'AMT_CREDIT_SUM_DEBT': ['max', 'mean', 'sum'],
    'AMT_CREDIT_SUM_OVERDUE': ['mean'],
    'AMT_CREDIT_SUM_LIMIT': ['mean', 'sum'],
    'AMT_ANNUITY': ['max', 'mean'],
    'CNT_CREDIT_PROLONG': ['sum'],
    'MONTHS_BALANCE_MIN': ['min'],
    'MONTHS_BALANCE_MAX': ['max'],
    'MONTHS_BALANCE_SIZE': ['mean', 'sum']
}
    
cat_aggregations = {}
for cat in bureau_cat: cat_aggregations[cat] = ['mean']
for cat in bb_cat: cat_aggregations[cat + "_MEAN"] = ['mean']

bureau_agg = bureau.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
bureau_agg.columns = pd.Index(['BURO_' + e[0] + "_" + e[1].upper() for e in bureau_agg.columns.tolist()])

# Créditos activos en bureau agregaciones de variables numéricas
active = bureau[bureau['CREDIT_ACTIVE_Active'] == 1]
active_agg = active.groupby('SK_ID_CURR').agg(num_aggregations)
active_agg.columns = pd.Index(['ACTIVE_' + e[0] + "_" + e[1].upper() for e in active_agg.columns.tolist()])
bureau_agg = bureau_agg.join(active_agg, how='left', on='SK_ID_CURR')
del active, active_agg

In [14]:
closed = bureau[bureau['CREDIT_ACTIVE_Closed'] == 1]
closed_agg = closed.groupby('SK_ID_CURR').agg(num_aggregations)
closed_agg.columns = pd.Index(['CLOSED_' + e[0] + "_" + e[1].upper() for e in closed_agg.columns.tolist()])
bureau_agg = bureau_agg.join(closed_agg, how='left', on='SK_ID_CURR')
del closed, closed_agg, bureau

### Transformaciones de previous_applications.csv 

In [15]:
prev = pd.read_csv('../data/previous_application.csv')

In [16]:
prev, cat_cols = one_hot_encoder(prev, nan_as_category= True)

In [17]:
# Sustitucion del valor 365243 por NA
prev['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace= True)
prev['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace= True)
prev['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace= True)
prev['DAYS_LAST_DUE'].replace(365243, np.nan, inplace= True)
prev['DAYS_TERMINATION'].replace(365243, np.nan, inplace= True)

# Nueva variable porcentaje del prestamo concedido sobre el pedido
prev['APP_CREDIT_PERC'] = prev['AMT_APPLICATION'] / prev['AMT_CREDIT']

num_aggregations = {
    'AMT_ANNUITY': ['min', 'max', 'mean'],
    'AMT_APPLICATION': ['min', 'max', 'mean'],
    'AMT_CREDIT': ['min', 'max', 'mean'],
    'APP_CREDIT_PERC': ['min', 'max', 'mean', 'var'],
    'AMT_DOWN_PAYMENT': ['min', 'max', 'mean'],
    'AMT_GOODS_PRICE': ['min', 'max', 'mean'],
    'HOUR_APPR_PROCESS_START': ['min', 'max', 'mean'],
    'RATE_DOWN_PAYMENT': ['min', 'max', 'mean'],
    'DAYS_DECISION': ['min', 'max', 'mean'],
    'CNT_PAYMENT': ['mean', 'sum'],
}

cat_aggregations = {}
for cat in cat_cols:
    cat_aggregations[cat] = ['mean']

prev_agg = prev.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
prev_agg.columns = pd.Index(['PREV_' + e[0] + "_" + e[1].upper() for e in prev_agg.columns.tolist()])

# Previous Applications agregaciones de las variables numéricas sobre las aplicaciones aprovadas
approved = prev[prev['NAME_CONTRACT_STATUS_Approved'] == 1]
approved_agg = approved.groupby('SK_ID_CURR').agg(num_aggregations)
approved_agg.columns = pd.Index(['APPROVED_' + e[0] + "_" + e[1].upper() for e in approved_agg.columns.tolist()])
prev_agg = prev_agg.join(approved_agg, how='left', on='SK_ID_CURR')

# Previous Applications agregaciones de las variables numéricas sobre las aplicaciones denegadas
refused = prev[prev['NAME_CONTRACT_STATUS_Refused'] == 1]
refused_agg = refused.groupby('SK_ID_CURR').agg(num_aggregations)
refused_agg.columns = pd.Index(['REFUSED_' + e[0] + "_" + e[1].upper() for e in refused_agg.columns.tolist()])
prev_agg = prev_agg.join(refused_agg, how='left', on='SK_ID_CURR')
del refused, refused_agg, approved, approved_agg, prev

### Transformaciones POS_CASH_balance.csv

In [18]:
pos = pd.read_csv('../data/POS_CASH_balance.csv')

In [19]:
pos, cat_cols = one_hot_encoder(pos, nan_as_category= True)

In [20]:
aggregations = {
    'MONTHS_BALANCE': ['max', 'mean', 'size'],
    'SK_DPD': ['max', 'mean'],
    'SK_DPD_DEF': ['max', 'mean']
}

for cat in cat_cols:
    aggregations[cat] = ['mean']

pos_agg = pos.groupby('SK_ID_CURR').agg(aggregations)
pos_agg.columns = pd.Index(['POS_' + e[0] + "_" + e[1].upper() for e in pos_agg.columns.tolist()])

# Numero de cuentas
pos_agg['POS_COUNT'] = pos.groupby('SK_ID_CURR').size()
del pos

### Transformaciones installments_payments.csv

In [21]:
ins = pd.read_csv('../data/installments_payments.csv')

In [22]:
ins, cat_cols = one_hot_encoder(ins, nan_as_category= True)

In [23]:
# Porcentaje pagado sobre la cuota mensual y diferencia entre la cuota y lo pagado
ins['PAYMENT_PERC'] = ins['AMT_PAYMENT'] / ins['AMT_INSTALMENT']
ins['PAYMENT_DIFF'] = ins['AMT_INSTALMENT'] - ins['AMT_PAYMENT']

# Days past due y days before due 
ins['DPD'] = ins['DAYS_ENTRY_PAYMENT'] - ins['DAYS_INSTALMENT']
ins['DBD'] = ins['DAYS_INSTALMENT'] - ins['DAYS_ENTRY_PAYMENT']

# Valores positivos
ins['DPD'] = ins['DPD'].apply(lambda x: x if x > 0 else 0)
ins['DBD'] = ins['DBD'].apply(lambda x: x if x > 0 else 0)

aggregations = {
    'NUM_INSTALMENT_VERSION': ['nunique'],
    'DPD': ['max', 'mean', 'sum'],
    'DBD': ['max', 'mean', 'sum'],
    'PAYMENT_PERC': ['max', 'mean', 'sum', 'var'],
    'PAYMENT_DIFF': ['max', 'mean', 'sum', 'var'],
    'AMT_INSTALMENT': ['max', 'mean', 'sum'],
    'AMT_PAYMENT': ['min', 'max', 'mean', 'sum'],
    'DAYS_ENTRY_PAYMENT': ['max', 'mean', 'sum']
}
for cat in cat_cols:
    aggregations[cat] = ['mean']
ins_agg = ins.groupby('SK_ID_CURR').agg(aggregations)
ins_agg.columns = pd.Index(['INSTAL_' + e[0] + "_" + e[1].upper() for e in ins_agg.columns.tolist()])

# Numero de cuentas en installments
ins_agg['INSTAL_COUNT'] = ins.groupby('SK_ID_CURR').size()
del ins

### Transformaciones credit_card_balance.csv

In [24]:
cc = pd.read_csv('../data/credit_card_balance.csv')

In [25]:
cc, cat_cols = one_hot_encoder(cc, nan_as_category= True)

In [26]:
cc.drop(['SK_ID_PREV'], axis= 1, inplace = True)
cc_agg = cc.groupby('SK_ID_CURR').agg(['min', 'max', 'mean', 'sum', 'var'])
cc_agg.columns = pd.Index(['CC_' + e[0] + "_" + e[1].upper() for e in cc_agg.columns.tolist()])

# Numero de tarjetas de crédito
cc_agg['CC_COUNT'] = cc.groupby('SK_ID_CURR').size()
del cc

### Union de los dataframes con la variable `SK_ID_CURR`

In [27]:
datos = datos.join(bureau_agg, how='left', on='SK_ID_CURR')
datos = datos.join(prev_agg, how='left', on='SK_ID_CURR')
datos = datos.join(pos_agg, how='left', on='SK_ID_CURR')
datos = datos.join(ins_agg, how='left', on='SK_ID_CURR')
datos = datos.join(cc_agg, how='left', on='SK_ID_CURR')
del bureau_agg, prev_agg, pos_agg, ins_agg, cc_agg

In [28]:
datos.drop('SK_ID_CURR', axis=1, inplace=True)

In [29]:
# Conversion de los nombres de las columnas con caracteres que no admite LightGBM
datos = datos.rename(columns = lambda x: x.replace(':', '').replace(',', ''))

In [30]:
# Elimino las variables que solo tienen un único valor
null_features = []

for var in datos.columns:
    if datos[f'{var}'].nunique() == 1:
        null_features.append(var)

In [31]:
len(null_features)

38

In [32]:
datos.drop(null_features, axis=1, inplace=True)

In [33]:
datos.to_csv('../data/datos_procesados.csv', index=False)